In [1]:
import warnings 
warnings.filterwarnings(action='ignore')

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import json
import time

from tensorflow.keras.models import load_model
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tqdm import tqdm_notebook

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. 전처리 완료된 데이터 불러오기

In [4]:
X_train = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/01_K-Fold_X/data/X_train.csv', index_col=[0])
X_test = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/01_K-Fold_X/data/X_test.csv', index_col=[0])
y_train = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/01_K-Fold_X/data/y_train.csv', index_col=[0])
y_test = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/01_K-Fold_X/data/y_test.csv', index_col=[0])

word_index_json = open('/content/drive/MyDrive/[final_project]_악플원정대/01_K-Fold_X/data/word_index_vocab.json', 'r').read()
word_index_vocab = json.loads(word_index_json)

In [5]:
X_train_split = X_train['댓글'].apply(list).tolist()
X_test_split = X_test['댓글'].apply(list).tolist()

## 2. 단어 사전을 이용한 토큰화와 패딩화

In [6]:
tokenizer = Tokenizer()
tokenizer.word_index = word_index_vocab

X_train_sequences = tokenizer.texts_to_sequences(X_train_split)
X_test_sequences = tokenizer.texts_to_sequences(X_test_split)

In [7]:
train = pad_sequences(X_train_sequences, padding='post', maxlen=400)
test = pad_sequences(X_test_sequences, padding='post', maxlen=400)

In [8]:
# 데이터가 추가되었는지 확인하고 진행하기 바람
# X_train.csv / X_test.csv / y_train.csv / y_test.csv 가 업데이트 되면 런타임을 재실행 한 후 불러와야 적용이됨
# 꼭 확인하고 진행하기를

y_train.value_counts()

악플여부
0       55729
1       32479
dtype: int64

## 3. 데이터의 불균형 해소 : SMOTE

In [69]:
# from imblearn.over_sampling import SMOTE

In [ ]:
# start = time.time()

# smote = SMOTE(random_state=123)
# train_over, y_train_over = smote.fit_sample(train, y_train)

# print("걸린시간 :", time.time() - start)

In [9]:
train_over = train
y_train_over = y_train

##4. 1D_CNN

In [10]:
import tensorflow.keras.backend as K
# Mish 활성화 함수
def mish(x):
    return x * K.tanh(K.softplus(x))

In [11]:
# 음절 단어사전의 크기
vocab_size = len(word_index_vocab)
embedding_dim = 32
comment_len = 400

In [23]:
def get_model() :

    from tensorflow.keras import Sequential
    from tensorflow.keras.layers import Dense, Embedding, GlobalMaxPooling1D, Conv1D, Dropout, Flatten, MaxPool1D, GlobalAveragePooling1D, Flatten

    model = Sequential()
    model.add(Embedding(vocab_size+1, embedding_dim, input_length=comment_len))
    model.add(Dropout(0.2))
    model.add(Conv1D(50, 3, padding='same', activation=mish, strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(50, activation=mish))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

    return model


In [19]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
mc = ModelCheckpoint('1D_CNN_best.h5', monitor='val_loss', mode='min', save_best_only=True, restore_best_weights=True)

In [24]:
model = get_model()
history = model.fit(train_over, y_train_over, 
                    callbacks        = [es, mc],
                    epochs           = 50,
                    batch_size       = 32, 
                    validation_data = (test, y_test))

Epoch 1/50
2757/2757 [==============================] - 109s 39ms/step - loss: 0.3135 - accuracy: 0.8738 - val_loss: 0.2526 - val_accuracy: 0.9024
Epoch 2/50
2757/2757 [==============================] - 111s 40ms/step - loss: 0.2540 - accuracy: 0.9020 - val_loss: 0.2234 - val_accuracy: 0.9121
Epoch 3/50
2757/2757 [==============================] - 108s 39ms/step - loss: 0.2406 - accuracy: 0.9069 - val_loss: 0.2265 - val_accuracy: 0.9165
Epoch 4/50
2757/2757 [==============================] - 110s 40ms/step - loss: 0.2312 - accuracy: 0.9114 - val_loss: 0.2179 - val_accuracy: 0.9193
Epoch 5/50
2757/2757 [==============================] - 104s 38ms/step - loss: 0.2247 - accuracy: 0.9141 - val_loss: 0.2356 - val_accuracy: 0.9156
Epoch 6/50
2757/2757 [==============================] - 103s 37ms/step - loss: 0.2215 - accuracy: 0.9155 - val_loss: 0.2367 - val_accuracy: 0.9108
Epoch 7/50
2757/2757 [==============================] - 108s 39ms/step - loss: 0.2181 - accuracy: 0.9172 - val_loss: 0

In [ ]:
# 저장한 CNN 모델을 불러오고 예측해보자

get_custom_objects().update({ 'mish' : mish })

loaded_model = load_model('1D_CNN_best.h5')
y_pred = loaded_model.predict(test)

y_class = y_pred > 0.5

In [ ]:
from sklearn.metrics import recall_score, precision_score, f1_score, roc_auc_score, confusion_matrix

print('recall : ', recall_score(y_class, y_test))
print('precision : ', precision_score(y_class, y_test))
print('f1_score : ', f1_score(y_class, y_test))
print('roc_auc_score : ', roc_auc_score(y_test, y_pred))
print(confusion_matrix(y_class, y_test))

## 5. 베타테스트 시행

In [ ]:
def model_test(comment):
    loaded_model = load_model('1D_CNN_best.h5')
    comment_list = [list(comment)]
    comment_label = tokenizer.texts_to_sequences(comment_list)

    comment_pad = pad_sequences(comment_label, padding='post', maxlen=400)

    pred = loaded_model.predict(comment_pad)

    if pred < 0.5 :
        result = '악플이 아닙니다'
    else :
        result = '악플입니다'
    
    print(result)
    print('라벨확률 : ', pred)
    
    return result, pred

In [ ]:
comment = ''''''

result, prob = model_test(comment)